In [1]:
import pandas as pd
import numpy as np
import gc

# Only read up to 4096 bytes, > 4096 has 100% malware rate
MAX_SIZE = 4096
TOTAL_ROWS =  113636
TOTAL_ROWS = 20000
USE_COLS = list(range(2, MAX_SIZE))
ROWS = TOTAL_ROWS

train = pd.read_csv("./data/train/train.zip", nrows=ROWS, usecols=USE_COLS, header=None, names = list(range(0, MAX_SIZE)), error_bad_lines=False)
train_label = pd.read_csv("./data/train_label.csv", usecols=[1], nrows=ROWS)

train = train.fillna(0, downcast='infer')
assert train.shape[0] == train_label.shape[0], "Train and label shapes are different"

In [2]:
import keras
from keras.layers import *
from keras import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras import optimizers
from keras import regularizers

mask = np.random.rand(len(train)) < 0.8

train_data = train.values
train_labels = train_label.values

x_train = train_data[mask]
y_train = train_labels[mask]
x_test = train_data[~mask]
y_test = train_labels[~mask]

Using TensorFlow backend.


In [3]:
train_labels = train_label.values

own_embedding_vocab_size = 256

maxlen = 4094

model = Sequential()
model.add(Embedding(input_dim=own_embedding_vocab_size, # 10
                    output_dim=32, 
                    input_length=maxlen))
model.add(Dropout(rate=0.2))

model.add(Flatten())

model.add(Dense(256, activation='elu', activity_regularizer=regularizers.l1_l2(0.0001)))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())

model.add(Dense(128, activation='elu', activity_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())

model.add(Dense(64, activation='elu', activity_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())

model.add(Dense(32, activation='elu', activity_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid', activity_regularizer=regularizers.l2(0.0001)))

adam=optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])  # Compile the model
print(model.summary())  # Summarize the model
model.fit(x_train, y_train, epochs=50, verbose=1)  # Fit the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)  # Evaluate the model
model.save('mlp_model.h5')
print('Accuracy: %0.3f' % accuracy)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4094, 32)          8192      
_________________________________________________________________
dropout_1 (Dropout)          (None, 4094, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 131008)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33538304  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
__________